In [2]:
import pandas as pd

df = pd.read_csv("ecommerceDataset.csv", names=["category", "description"], header=None)
df.head()

,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [3]:
df.shape

(50425, 2)

In [4]:
df.isna().sum()

category       0
description    1
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.shape

(50424, 2)

In [7]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [8]:
df.category.replace('Clothing & Accessories', 'Clothing_Accessories', inplace=True)

In [9]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [10]:
df['category'] = '__label__' + df['category'].astype(str)
df.head()

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [11]:
df['category_description'] = df['category'] + " " + df['description']
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


### Preprocessing

In [12]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r"[^\s\w\']", ' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [13]:
def preprocess(text):
    text = re.sub(r"[^\s\w\']", ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [14]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


### Train test split

In [18]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [19]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [26]:
train.to_csv("ecommerce.train", columns=['category_description'], index=False, header=False)
test.to_csv("ecommerce.test", columns=['category_description'], index=False, header=False)

### Train the model

In [27]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")

In [28]:
model.test("ecommerce.test")

(10083, 0.9706436576415749, 0.9706436576415749)

### Prediction

In [29]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.98790938]))

In [30]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [31]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00001001]))

In [32]:
model.get_nearest_neighbors("painting")

[(0.9985708594322205, 'picoting'),
 (0.9985117316246033, 'snowbird'),
 (0.9985117316246033, 'kungfu'),
 (0.9985033273696899, "natureloc's"),
 (0.9984548687934875, 'pevb223'),
 (0.9984510540962219, 'lebedo'),
 (0.9984509348869324, 'hernias'),
 (0.9984509348869324, 'hiatal'),
 (0.9984343647956848, 'zjchao'),
 (0.9984343647956848, 'rattail')]